- https://myapollo.com.tw/blog/langchain-tutorial-agent/

In [8]:
import requests
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import tool
from langchain.agents import AgentExecutor

In [3]:
from dotenv import load_dotenv
assert load_dotenv()

In [9]:
os.environ["LANGCHAIN_PROJECT"] = "0924"

In [10]:
@tool
def check_site_alive(site: str) -> bool:
    """Check a site is alive or not."""
    try:
        resp = requests.get(f'https://{site}')
        resp.raise_for_status()
        return True
    except Exception:
        return False

In [11]:
tools = [check_site_alive, ]


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [12]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are very powerful assistant, but don't know current events",
    ),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

- `x["intermediate_steps"]` 是由 AgentExecutor 产生并给到 agent 执行的；
-  AgentExecutor 把 agent 与 tools 组成 Agent Executor;
-  The agent executor is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats. 

In [17]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable={
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are very powerful assistant, but don't know current events")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f5a4c163920>, async_client=<openai.resources.chat.comp

In [13]:
response = agent_executor.invoke({
    'input': 'is example.com alive?',
})



> Entering new AgentExecutor chain...

Invoking: `check_site_alive` with `{'site': 'example.com'}`


TrueYes, example.com is alive.

> Finished chain.


In [14]:
response

{'input': 'is example.com alive?', 'output': 'Yes, example.com is alive.'}